In [1]:
import pandas as pd
import numpy as np
from h3 import h3
import geojson
import folium
from folium import plugins
import webbrowser
import geopandas as gpd
import shapely.geometry
from shapely.geometry import MultiPolygon,Polygon,LineString
from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj
from helper.pandas_helper import *
from helper.presto_helper import *
import ot

/home/ai.liuxuanheng/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/ai.liuxuanheng/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
station_df = pd.read_csv('360station.csv')
bike_1_df = pd.read_csv('360bike1.csv')
bike_2_df = pd.read_csv('360bike2.csv')
bike_3_df = pd.read_csv('360bike3.csv')
bike_df = pd.concat([bike_1_df,bike_2_df,bike_3_df],axis=0).reset_index(drop=True)
print(station_df.shape,bike_df.shape)

(330, 23) (1206, 21)


# 多尺度机会成本

In [60]:
oppo_df = get_df_from_csv_table(query_presto(f"""
select
  city_id,block_id,opportunity_cost
from
  ai.dws_etp_block_opportunity_cost_v2_da
where
  event_day = '20220902'
  and model_version= '1'
  and city_id = 360
  and span = '8~23'
"""))
oppo_df.to_csv('360oppo_df.csv')

# block

In [6]:
block_df = bike_df.groupby(['city_id','block_id','block_center_point']).agg({'bike_sn':'count'}).reset_index()
block_df = block_df.merge(oppo_df,on=['city_id','block_id'],how='inner')
block_df = block_df.sort_values(['block_id']).reset_index(drop=True)

# station

In [7]:
station_df = station_df[['city_id','station_id', 'station_center_point', 'block_id', 'target_bike_cnt',
       'bike_num', 'predict_in', 'dispatch_num', 'efficiency', 'profit_order',
       'in_bike_cnt', 'explore_target_bike_cnt']]
station_df['station_profit'] = station_df.apply(lambda x:0 if (x['target_bike_cnt']-x['bike_num']-x['predict_in'])<=0 else x['profit_order'],axis=1)

def get_target(x):
    if x['target_bike_cnt']-max(x['predict_in'],x['in_bike_cnt'])-x['dispatch_num']>=0: #时段初期需求正常
        tg = x['target_bike_cnt']-max(x['predict_in'],x['in_bike_cnt'])-x['dispatch_num']
    else:
        if x['predict_in']>x['in_bike_cnt']:  # 时段初期需求正常但预测流入偏高/末期需求减少
            tg = x['target_bike_cnt']
        else:      # 时段末期需求突增
            tg = int(0.8*x['target_bike_cnt'])
    return tg

station_df['target_bike_cnt_revise'] = station_df.apply(lambda x: get_target(x),axis=1)
station_df = station_df.sort_values(['station_id']).reset_index(drop=True)
station_df.rename(columns={'block_id':'station_block_id'},inplace=True)
filter_1_station_list = list(station_df[station_df.target_bike_cnt_revise==1]['station_id'])
station_df['target_bike_cnt_revise'] = station_df.target_bike_cnt_revise.map(lambda x:0 if x==1 else x)
station_df = station_df.sort_values(['station_id']).reset_index(drop=True)

print(station_df.shape)
station_df.head()

(330, 14)


,city_id,station_id,station_center_point,station_block_id,target_bike_cnt,bike_num,predict_in,dispatch_num,efficiency,profit_order,in_bike_cnt,explore_target_bike_cnt,station_profit,target_bike_cnt_revise
0,360,35333,118.11166529765|36.964322593158,8b30a4a9a085fff,6,3,5,0,0,1.378790,9,6,0.000000,4
1,360,35346,118.11335411201|36.975711604558,8b30a4a984a2fff,4,0,2,0,0,0.706414,1,0,0.706414,2
2,360,35347,118.10033766517|36.976435974051,8b30a4a9e620fff,5,9,3,0,0,0.794813,1,3,0.000000,2
3,360,35348,118.09285028874|36.974774598895,8b30a4a9e4d6fff,3,2,1,0,0,1.120420,1,5,0.000000,2
4,360,35349,118.09263469542|36.969638880437,8b30a406db9bfff,9,0,3,0,0,1.378190,2,10,1.378190,6


# cost

In [8]:
fuse_df = block_df.merge(station_df,on=['city_id'],how ='inner')
def get_dis(x):
    s_lon,s_lat = float(x['station_center_point'].split('|')[0]),float(x['station_center_point'].split('|')[1])
    b_lon,b_lat = float(x['block_center_point'].split('|')[0]), float(x['block_center_point'].split('|')[1])
    dis = 1000 * 6371.393 * np.arccos(
    np.cos(np.radians(s_lat)) * np.cos(np.radians(b_lat)) * np.cos(np.radians(b_lon)-np.radians(s_lon))
    + np.sin(np.radians(b_lat)) * np.sin(np.radians(s_lat))  )
    return dis
def get_dis_cost(x):
    if 0<=x<=1000:
        cost = (1.4-0)*(x-0)/(1000-0)
    elif 1000<x<=2500:
        cost = (2.1-1.4)*(x-1000)/(2500-1000) + 1.4
    elif 2500<x<=4000:
        cost = (2.8-2.1)*(x-2500)/(4000-2500) + 2.1
    elif 4000<x<=5500:
        cost = (4.2-2.8)*(x-4000)/(5500-4000) + 2.8
    else:
        cost = 4.2
    return cost

fuse_df['dis'] = fuse_df.apply(lambda x:get_dis(x),axis=1)
fuse_df['dis'] = fuse_df.apply(lambda x:0 if x.block_id==x.station_block_id else x.dis, axis=1)
fuse_df['dis_cost'] = fuse_df.apply(lambda x:get_dis_cost(x['dis']),axis=1)

fuse_df['op9'] = fuse_df.opportunity_cost.map(lambda x:eval(x)['9'])
fuse_df['op10'] = fuse_df.opportunity_cost.map(lambda x:eval(x)['10'])
fuse_df['op11'] = fuse_df.opportunity_cost.map(lambda x:eval(x)['11'])

# fuse

In [9]:
filter_1_block_list=list(fuse_df[(fuse_df['dis']<1) & (fuse_df['station_id'].isin(filter_1_station_list))]['block_id'])
block_df['bikes'] = block_df.apply(lambda x:x['bike_sn']-1 if x['block_id'] in filter_1_block_list and x['bike_sn']>0 else x['bike_sn'],axis=1)

In [10]:
fuse_df['total_profit'] = fuse_df.apply(lambda x:999 if x['dis']< 25 else (x['station_profit']-x['op11'])*2.5-x['dis_cost'] ,axis=1)
fuse_df['end_cost'] = -fuse_df.total_profit
cost_array = np.reshape(np.array(list(fuse_df['end_cost'])),(-1,len(station_df))) # b --> s


In [11]:
#=== calcute ===
b = list(block_df['bikes'])
s = list(station_df['target_bike_cnt_revise'])
print("supply accept:",sum(b),sum(s))
m = min(sum(s),sum(b))
M = cost_array
martix = ot.partial.partial_wasserstein(b,s,M=cost_array,m=m)

va2 = pd.DataFrame(martix,index=list(block_df['block_id']),columns =list(station_df['station_id']))
res2 = []
for idx,val in va2.iterrows():
    for col in va2.columns:
        if val[col] > 0:
            res2.append((idx,col,val[col]))
e1 = pd.DataFrame(data=res2,columns=['block_id','station_id','nums'])
e2 = e1.merge(block_df,on=['block_id'],how='inner')
e3 = e2.merge(station_df,on=['station_id'],how='inner')
def get_lstring(x):
    slon,slat = float(x['block_center_point'].split('|')[0]), float(x['block_center_point'].split('|')[1])
    elon,elat = float(x['station_center_point'].split('|')[0]), float(x['station_center_point'].split('|')[1])
    s_lon,s_lat = bd2gcj(slon,slat)
    e_lon,e_lat = bd2gcj(elon,elat)
    return LineString([[s_lon,s_lat],[e_lon,e_lat]])
e3['Linestring'] = e3.apply(lambda x:get_lstring(x), axis=1)
e3['geo_json'] = e3.apply(lambda x:geojson.FeatureCollection(features=[geojson.Feature(geometry=x['Linestring'], properties={'xxxxxx':1})]),axis=1)

supply accept: 1148 400


In [12]:

"""
create map
"""
m = folium.Map(location=[36.970453,118.107313],zoom_start=12,
        tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}', # 高德街道图
#         tiles='http://webst02.is.autonavi.com/appmaptile?style=6&x={x}&y={y}&z={z}', # 高德卫星图
#         tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', # google 卫星图
#         tiles='https://mt.google.com/vt/lyrs=h&x={x}&y={y}&z={z}', # google 地图
        attr='default')
m.add_child(plugins.MeasureControl())
"""
add move bike  
"""
for idx,df in e3.iterrows():
    elon,elat = float(df['station_center_point'].split('|')[0]), float(df['station_center_point'].split('|')[1])
    e_lon,e_lat = bd2gcj(elon,elat)
    folium.Circle(
        location=[e_lat,e_lon],
      radius=25,
      color= 'red'
    ).add_to(m)
    folium.GeoJson(
        df['geo_json'],
        style_function=lambda feature: {
            'popup': "",
            'fillColor': 'blue',
            'color': 'green',
            'weight': 2
#             'dashArray': '5, 5'
        }
    ).add_to(m)
m

# 多尺度h3
- 离车站越近用颗粒度小的h3等级
- 用每个区块距离所有车站的最近距离，区分远近

In [13]:
fuse_df2 = fuse_df.copy()
block_min_dis = fuse_df2.groupby(['block_id']).dis.apply(lambda x:min(x)).reset_index()
block_min_dis.describe()

,dis
count,392.000000
mean,52.604675
std,110.642659
min,0.000000
25%,0.000000
50%,23.963740
75%,39.550488
max,938.326180


In [14]:
b_0_50 = list(block_min_dis[block_min_dis.dis<23].block_id)
b_50_100 = list(block_min_dis[(block_min_dis.dis>=23) & (block_min_dis.dis<39)].block_id)
b_100_999 = list(block_min_dis[block_min_dis.dis>=39].block_id)
def get_op(x):
    if x['block_id'] in b_0_50:
        op = x['op11']
    elif x['block_id'] in b_50_100:
        op = x['op10']
    else:
        op = x['op9']
    return op
fuse_df2['op_tmp'] = fuse_df2.apply(lambda x:get_op(x),axis=1)

fuse_df2['total_profit'] = fuse_df2.apply(lambda x:999 if x['dis']< 25 else (x['station_profit']-x['op_tmp'])*2.5-x['dis_cost'] ,axis=1)
fuse_df2['end_cost'] = -fuse_df2.total_profit
cost_array = np.reshape(np.array(list(fuse_df2['end_cost'])),(-1,len(station_df))) # b --> s


In [15]:
#=== calcute ===
b = list(block_df['bikes'])
s = list(station_df['target_bike_cnt_revise'])
print("supply accept:",sum(b),sum(s))
m = min(sum(s),sum(b))
M = cost_array
martix = ot.partial.partial_wasserstein(b,s,M=cost_array,m=m)

va2 = pd.DataFrame(martix,index=list(block_df['block_id']),columns =list(station_df['station_id']))
res3 = []
for idx,val in va2.iterrows():
    for col in va2.columns:
        if val[col] > 0:
            res3.append((idx,col,val[col]))
f1 = pd.DataFrame(data=res3,columns=['block_id','station_id','nums'])
f2 = f1.merge(block_df,on=['block_id'],how='inner')
f3 = f2.merge(station_df,on=['station_id'],how='inner')
def get_lstring(x):
    slon,slat = float(x['block_center_point'].split('|')[0]), float(x['block_center_point'].split('|')[1])
    elon,elat = float(x['station_center_point'].split('|')[0]), float(x['station_center_point'].split('|')[1])
    s_lon,s_lat = bd2gcj(slon,slat)
    e_lon,e_lat = bd2gcj(elon,elat)
    return LineString([[s_lon,s_lat],[e_lon,e_lat]])
f3['Linestring'] = f3.apply(lambda x:get_lstring(x), axis=1)
f3['geo_json'] = f3.apply(lambda x:geojson.FeatureCollection(features=[geojson.Feature(geometry=x['Linestring'], properties={'xxxxxx':1})]),axis=1)

supply accept: 1148 400


In [16]:

"""
create map
"""

m = folium.Map(location=[36.970453,118.107313],zoom_start=12,
        tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}', # 高德街道图
#         tiles='http://webst02.is.autonavi.com/appmaptile?style=6&x={x}&y={y}&z={z}', # 高德卫星图
#         tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', # google 卫星图
#         tiles='https://mt.google.com/vt/lyrs=h&x={x}&y={y}&z={z}', # google 地图
        attr='default')
m.add_child(plugins.MeasureControl())
"""
add move bike  
"""
for idx,df in f3.iterrows():
    elon,elat = float(df['station_center_point'].split('|')[0]), float(df['station_center_point'].split('|')[1])
    e_lon,e_lat = bd2gcj(elon,elat)
    folium.Circle(
        location=[e_lat,e_lon],
      radius=25,
      color= 'red'
    ).add_to(m)
    folium.GeoJson(
        df['geo_json'],
        style_function=lambda feature: {
            'popup': "",
            'fillColor': 'blue',
            'color': 'green',
            'weight': 2
#             'dashArray': '5, 5'
        }
    ).add_to(m)

m

In [17]:
len(res2)

232

In [18]:
len(res3)

199

# 多尺度2
- 找11级的10，9级，分组聚合求平均得到机会成本
- 无需工程组改动的最优传输情况

In [20]:
fuse_df3 = fuse_df.copy()
fuse_df3.head()

,city_id,block_id,block_center_point,bike_sn,opportunity_cost,station_id,station_center_point,station_block_id,target_bike_cnt,bike_num,...,explore_target_bike_cnt,station_profit,target_bike_cnt_revise,dis,dis_cost,op9,op10,op11,total_profit,end_cost
0,360,8b30a4041b90fff,118.06814985715047|36.92208727457503,1,"{'9': 1.164613962173462, '10': 1.1646139621734...",35333,118.11166529765|36.964322593158,8b30a4a9a085fff,6,3,...,6,0.000000,4,6084.064159,4.2,1.164614,1.164614,0.0,-4.200000,4.200000
1,360,8b30a4041b90fff,118.06814985715047|36.92208727457503,1,"{'9': 1.164613962173462, '10': 1.1646139621734...",35346,118.11335411201|36.975711604558,8b30a4a984a2fff,4,0,...,0,0.706414,2,7190.082797,4.2,1.164614,1.164614,0.0,-2.433965,2.433965
2,360,8b30a4041b90fff,118.06814985715047|36.92208727457503,1,"{'9': 1.164613962173462, '10': 1.1646139621734...",35347,118.10033766517|36.976435974051,8b30a4a9e620fff,5,9,...,3,0.000000,2,6686.434875,4.2,1.164614,1.164614,0.0,-4.200000,4.200000
3,360,8b30a4041b90fff,118.06814985715047|36.92208727457503,1,"{'9': 1.164613962173462, '10': 1.1646139621734...",35348,118.09285028874|36.974774598895,8b30a4a9e4d6fff,3,2,...,5,0.000000,2,6256.641947,4.2,1.164614,1.164614,0.0,-4.200000,4.200000
4,360,8b30a4041b90fff,118.06814985715047|36.92208727457503,1,"{'9': 1.164613962173462, '10': 1.1646139621734...",35349,118.09263469542|36.969638880437,8b30a406db9bfff,9,0,...,10,1.378190,6,5718.060788,4.2,1.164614,1.164614,0.0,-0.754525,0.754525


In [48]:
tmp_h3_cost =  fuse_df3.drop_duplicates(['block_id'])[['block_id','op11']].reset_index(drop=True)
tmp_h3_cost['block_10'] = tmp_h3_cost.block_id.apply(lambda x:h3.h3_to_parent(x,10))
tmp_h3_cost['block_9'] = tmp_h3_cost.block_id.apply(lambda x:h3.h3_to_parent(x,9))
tmp_10 = tmp_h3_cost.groupby(['block_10']).agg({'op11':'mean'}).rename({'op11':'op10_new'},axis=1).reset_index()
tmp_9 = tmp_h3_cost.groupby(['block_9']).agg({'op11':'mean'}).rename({'op11':'op9_new'},axis=1).reset_index()

In [49]:
tmp_9

,block_9,op9_new
0,8930a4041bbffff,0.000000
1,8930a40613bffff,0.000000
2,8930a406147ffff,0.136152
3,8930a406187ffff,0.000000
4,8930a4061a7ffff,0.000000
...,...,...
157,8930a4a9e63ffff,0.338634
158,8930a4a9e6bffff,0.472868
159,8930a4a9e77ffff,0.247816
160,8930a4a9e7bffff,0.435511


In [50]:
fuse_df3['block_10'] = fuse_df3.block_id.apply(lambda x:h3.h3_to_parent(x,10))
fuse_df3['block_9'] = fuse_df3.block_id.apply(lambda x:h3.h3_to_parent(x,9))

In [51]:
r1 = fuse_df3.merge(tmp_10,on =['block_10'],how = 'left')
r2 = r1.merge(tmp_9,on =['block_9'],how = 'left')

In [52]:
block_min_dis = fuse_df3.groupby(['block_id']).dis.apply(lambda x:min(x)).reset_index()
block_min_dis.describe()

,dis
count,392.000000
mean,52.604675
std,110.642659
min,0.000000
25%,0.000000
50%,23.963740
75%,39.550488
max,938.326180


In [54]:
fuse_df3.columns

Index(['city_id', 'block_id', 'block_center_point', 'bike_sn',
       'opportunity_cost', 'station_id', 'station_center_point',
       'station_block_id', 'target_bike_cnt', 'bike_num', 'predict_in',
       'dispatch_num', 'efficiency', 'profit_order', 'in_bike_cnt',
       'explore_target_bike_cnt', 'station_profit', 'target_bike_cnt_revise',
       'dis', 'dis_cost', 'op9', 'op10', 'op11', 'total_profit', 'end_cost',
       'block_10', 'block_9'],
      dtype='object')

In [55]:
b_0_50 = list(block_min_dis[block_min_dis.dis<23].block_id)
b_50_100 = list(block_min_dis[(block_min_dis.dis>=23) & (block_min_dis.dis<39)].block_id)
b_100_999 = list(block_min_dis[block_min_dis.dis>=39].block_id)
def get_op(x):
    if x['block_id'] in b_0_50:
        op = x['op11']
    elif x['block_id'] in b_50_100:
        op = x['op10_new']
    else:
        op = x['op9_new']
    return op
r2['op_tmp'] = r2.apply(lambda x:get_op(x),axis=1)

r2['total_profit'] = r2.apply(lambda x:999 if x['dis']< 25 else (x['station_profit']-x['op_tmp'])*2.5-x['dis_cost'] ,axis=1)
r2['end_cost'] = -r2.total_profit
cost_array = np.reshape(np.array(list(r2['end_cost'])),(-1,len(station_df))) # b --> s


In [56]:
#=== calcute ===
b = list(block_df['bikes'])
s = list(station_df['target_bike_cnt_revise'])
print("supply accept:",sum(b),sum(s))
m = min(sum(s),sum(b))
M = cost_array
martix = ot.partial.partial_wasserstein(b,s,M=cost_array,m=m)

va2 = pd.DataFrame(martix,index=list(block_df['block_id']),columns =list(station_df['station_id']))
res4 = []
for idx,val in va2.iterrows():
    for col in va2.columns:
        if val[col] > 0:
            res4.append((idx,col,val[col]))
f1 = pd.DataFrame(data=res4,columns=['block_id','station_id','nums'])
f2 = f1.merge(block_df,on=['block_id'],how='inner')
f3 = f2.merge(station_df,on=['station_id'],how='inner')
def get_lstring(x):
    slon,slat = float(x['block_center_point'].split('|')[0]), float(x['block_center_point'].split('|')[1])
    elon,elat = float(x['station_center_point'].split('|')[0]), float(x['station_center_point'].split('|')[1])
    s_lon,s_lat = bd2gcj(slon,slat)
    e_lon,e_lat = bd2gcj(elon,elat)
    return LineString([[s_lon,s_lat],[e_lon,e_lat]])
f3['Linestring'] = f3.apply(lambda x:get_lstring(x), axis=1)
f3['geo_json'] = f3.apply(lambda x:geojson.FeatureCollection(features=[geojson.Feature(geometry=x['Linestring'], properties={'xxxxxx':1})]),axis=1)

supply accept: 1148 400


In [57]:

"""
create map
"""

m = folium.Map(location=[36.970453,118.107313,zoom_start=12,
        tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}', # 高德街道图
#         tiles='http://webst02.is.autonavi.com/appmaptile?style=6&x={x}&y={y}&z={z}', # 高德卫星图
#         tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', # google 卫星图
#         tiles='https://mt.google.com/vt/lyrs=h&x={x}&y={y}&z={z}', # google 地图
        attr='default')
m.add_child(plugins.MeasureControl())
"""
add move bike  
"""
for idx,df in f3.iterrows():
    elon,elat = float(df['station_center_point'].split('|')[0]), float(df['station_center_point'].split('|')[1])
    e_lon,e_lat = bd2gcj(elon,elat)
    folium.Circle(
        location=[e_lat,e_lon],
      radius=25,
      color= 'red'
    ).add_to(m)
    folium.GeoJson(
        df['geo_json'],
        style_function=lambda feature: {
            'popup': "",
            'fillColor': 'blue',
            'color': 'green',
            'weight': 2
#             'dashArray': '5, 5'
        }
    ).add_to(m)

m